In [ ]:
import React, { useState } from 'react';
import { motion, AnimatePresence } from 'framer-motion';
import { Card, CardContent, CardHeader, CardTitle } from "@/components/ui/card";
import { Button } from "@/components/ui/button";
import { Input } from "@/components/ui/input";
import { Label } from "@/components/ui/label";
import { Badge } from "@/components/ui/badge";
import { ChevronRight, ChevronLeft, Target, Utensils, AlertCircle, Activity, User } from 'lucide-react';
import { cn } from "@/lib/utils";

export default function ProfileSetup({ onComplete }) {
  const [step, setStep] = useState(1);
  const [profile, setProfile] = useState({
    dietary_goal: '',
    diet_type: 'none',
    allergies: [],
    weight: '',
    height: '',
    age: '',
    activity_level: '',
    daily_calorie_target: 2000,
    daily_protein_target: 150,
    daily_carbs_target: 250,
    daily_fats_target: 65
  });

  const goals = [
    { value: 'weight_loss', label: 'Weight Loss', icon: '🎯', desc: 'Lose weight healthily' },
    { value: 'muscle_gain', label: 'Muscle Gain', icon: '💪', desc: 'Build lean muscle' },
    { value: 'maintenance', label: 'Maintenance', icon: '⚖️', desc: 'Maintain current weight' },
    { value: 'health_improvement', label: 'Health', icon: '❤️', desc: 'Improve overall health' }
  ];

  const diets = [
    { value: 'none', label: 'No Preference' },
    { value: 'vegan', label: 'Vegan' },
    { value: 'vegetarian', label: 'Vegetarian' },
    { value: 'keto', label: 'Keto' },
    { value: 'paleo', label: 'Paleo' },
    { value: 'mediterranean', label: 'Mediterranean' },
    { value: 'low_carb', label: 'Low Carb' },
    { value: 'diabetic_friendly', label: 'Diabetic Friendly' }
  ];

  const commonAllergies = ['Peanuts', 'Tree Nuts', 'Dairy', 'Eggs', 'Soy', 'Wheat', 'Fish', 'Shellfish'];
  
  const activityLevels = [
    { value: 'sedentary', label: 'Sedentary', desc: 'Little or no exercise' },
    { value: 'light', label: 'Light', desc: '1-3 days/week' },
    { value: 'moderate', label: 'Moderate', desc: '3-5 days/week' },
    { value: 'active', label: 'Active', desc: '6-7 days/week' },
    { value: 'very_active', label: 'Very Active', desc: 'Intense daily exercise' }
  ];

  const toggleAllergy = (allergy) => {
    setProfile(prev => ({
      ...prev,
      allergies: prev.allergies.includes(allergy)
        ? prev.allergies.filter(a => a !== allergy)
        : [...prev.allergies, allergy]
    }));
  };

  const calculateTargets = () => {
    const weight = parseFloat(profile.weight);
    const height = parseFloat(profile.height);
    const age = parseFloat(profile.age);
    
    if (!weight || !height || !age) return;

    // Basic BMR calculation (Mifflin-St Jeor)
    let bmr = 10 * weight + 6.25 * height - 5 * age + 5;
    
    const activityMultipliers = {
      sedentary: 1.2, light: 1.375, moderate: 1.55, active: 1.725, very_active: 1.9
    };
    
    let tdee = bmr * (activityMultipliers[profile.activity_level] || 1.5);
    
    if (profile.dietary_goal === 'weight_loss') tdee -= 500;
    if (profile.dietary_goal === 'muscle_gain') tdee += 300;
    
    setProfile(prev => ({
      ...prev,
      daily_calorie_target: Math.round(tdee),
      daily_protein_target: Math.round(weight * 2),
      daily_carbs_target: Math.round((tdee * 0.45) / 4),
      daily_fats_target: Math.round((tdee * 0.3) / 9)
    }));
  };

  const handleNext = () => {
    if (step === 4) {
      calculateTargets();
      setTimeout(() => onComplete({ ...profile, onboarding_complete: true }), 100);
    } else {
      setStep(step + 1);
    }
  };

  const canProceed = () => {
    if (step === 1) return profile.dietary_goal;
    if (step === 2) return true;
    if (step === 3) return true;
    if (step === 4) return profile.weight && profile.height && profile.age && profile.activity_level;
    return false;
  };

  return (
    <div className="min-h-screen bg-gradient-to-br from-emerald-50 via-teal-50 to-cyan-50 p-6 flex items-center justify-center">
      <div className="w-full max-w-2xl">
        {/* Progress Bar */}
        <div className="mb-8">
          <div className="flex justify-between mb-2">
            <span className="text-sm font-medium text-slate-700">Step {step} of 4</span>
            <span className="text-sm text-slate-500">{Math.round((step / 4) * 100)}%</span>
          </div>
          <div className="h-2 bg-slate-200 rounded-full overflow-hidden">
            <motion.div 
              className="h-full bg-gradient-to-r from-emerald-500 to-teal-600"
              initial={{ width: 0 }}
              animate={{ width: `${(step / 4) * 100}%` }}
              transition={{ duration: 0.3 }}
            />
          </div>
        </div>

        <AnimatePresence mode="wait">
          {/* Step 1: Goal */}
          {step === 1 && (
            <motion.div
              key="step1"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-emerald-100 rounded-xl">
                      <Target className="w-6 h-6 text-emerald-600" />
                    </div>
                    <CardTitle className="text-2xl">What's your goal?</CardTitle>
                  </div>
                  <p className="text-slate-600">Help us personalize your experience</p>
                </CardHeader>
                <CardContent className="grid grid-cols-1 md:grid-cols-2 gap-4">
                  {goals.map((goal) => (
                    <motion.button
                      key={goal.value}
                      whileHover={{ scale: 1.02 }}
                      whileTap={{ scale: 0.98 }}
                      onClick={() => setProfile({ ...profile, dietary_goal: goal.value })}
                      className={cn(
                        "p-6 rounded-2xl border-2 text-left transition-all",
                        profile.dietary_goal === goal.value
                          ? "border-emerald-500 bg-emerald-50"
                          : "border-slate-200 hover:border-slate-300"
                      )}
                    >
                      <div className="text-4xl mb-3">{goal.icon}</div>
                      <h3 className="font-semibold text-lg mb-1">{goal.label}</h3>
                      <p className="text-sm text-slate-600">{goal.desc}</p>
                    </motion.button>
                  ))}
                </CardContent>
              </Card>
            </motion.div>
          )}

          {/* Step 2: Diet Type */}
          {step === 2 && (
            <motion.div
              key="step2"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-teal-100 rounded-xl">
                      <Utensils className="w-6 h-6 text-teal-600" />
                    </div>
                    <CardTitle className="text-2xl">Dietary preference?</CardTitle>
                  </div>
                  <p className="text-slate-600">Optional - helps with food recommendations</p>
                </CardHeader>
                <CardContent className="grid grid-cols-2 md:grid-cols-3 gap-3">
                  {diets.map((diet) => (
                    <Button
                      key={diet.value}
                      variant={profile.diet_type === diet.value ? "default" : "outline"}
                      onClick={() => setProfile({ ...profile, diet_type: diet.value })}
                      className={cn(
                        "h-auto py-4",
                        profile.diet_type === diet.value && "bg-teal-600 hover:bg-teal-700"
                      )}
                    >
                      {diet.label}
                    </Button>
                  ))}
                </CardContent>
              </Card>
            </motion.div>
          )}

          {/* Step 3: Allergies */}
          {step === 3 && (
            <motion.div
              key="step3"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-red-100 rounded-xl">
                      <AlertCircle className="w-6 h-6 text-red-600" />
                    </div>
                    <CardTitle className="text-2xl">Any allergies?</CardTitle>
                  </div>
                  <p className="text-slate-600">We'll alert you when scanning food</p>
                </CardHeader>
                <CardContent className="space-y-4">
                  <div className="flex flex-wrap gap-2">
                    {commonAllergies.map((allergy) => (
                      <Badge
                        key={allergy}
                        variant={profile.allergies.includes(allergy) ? "default" : "outline"}
                        className={cn(
                          "cursor-pointer py-2 px-4 text-sm",
                          profile.allergies.includes(allergy) && "bg-red-600 hover:bg-red-700"
                        )}
                        onClick={() => toggleAllergy(allergy)}
                      >
                        {allergy}
                      </Badge>
                    ))}
                  </div>
                  <p className="text-xs text-slate-500">Tap to select/deselect</p>
                </CardContent>
              </Card>
            </motion.div>
          )}

          {/* Step 4: Stats */}
          {step === 4 && (
            <motion.div
              key="step4"
              initial={{ opacity: 0, x: 20 }}
              animate={{ opacity: 1, x: 0 }}
              exit={{ opacity: 0, x: -20 }}
            >
              <Card className="border-0 shadow-xl">
                <CardHeader>
                  <div className="flex items-center gap-3 mb-2">
                    <div className="p-3 bg-purple-100 rounded-xl">
                      <User className="w-6 h-6 text-purple-600" />
                    </div>
                    <CardTitle className="text-2xl">About you</CardTitle>
                  </div>
                  <p className="text-slate-600">To calculate your daily targets</p>
                </CardHeader>
                <CardContent className="space-y-6">
                  <div className="grid grid-cols-3 gap-4">
                    <div>
                      <Label htmlFor="weight">Weight (kg)</Label>
                      <Input
                        id="weight"
                        type="number"
                        value={profile.weight}
                        onChange={(e) => setProfile({ ...profile, weight: e.target.value })}
                        placeholder="70"
                        className="mt-2"
                      />
                    </div>
                    <div>
                      <Label htmlFor="height">Height (cm)</Label>
                      <Input
                        id="height"
                        type="number"
                        value={profile.height}
                        onChange={(e) => setProfile({ ...profile, height: e.target.value })}
                        placeholder="170"
                        className="mt-2"
                      />
                    </div>
                    <div>
                      <Label htmlFor="age">Age</Label>
                      <Input
                        id="age"
                        type="number"
                        value={profile.age}
                        onChange={(e) => setProfile({ ...profile, age: e.target.value })}
                        placeholder="30"
                        className="mt-2"
                      />
                    </div>
                  </div>

                  <div>
                    <Label className="mb-3 block">Activity Level</Label>
                    <div className="space-y-2">
                      {activityLevels.map((level) => (
                        <button
                          key={level.value}
                          onClick={() => setProfile({ ...profile, activity_level: level.value })}
                          className={cn(
                            "w-full p-4 rounded-xl border-2 text-left transition-all",
                            profile.activity_level === level.value
                              ? "border-purple-500 bg-purple-50"
                              : "border-slate-200 hover:border-slate-300"
                          )}
                        >
                          <div className="flex justify-between items-center">
                            <div>
                              <p className="font-semibold">{level.label}</p>
                              <p className="text-sm text-slate-600">{level.desc}</p>
                            </div>
                            {profile.activity_level === level.value && (
                              <Activity className="w-5 h-5 text-purple-600" />
                            )}
                          </div>
                        </button>
                      ))}
                    </div>
                  </div>
                </CardContent>
              </Card>
            </motion.div>
          )}
        </AnimatePresence>

        {/* Navigation */}
        <div className="flex gap-3 mt-6">
          {step > 1 && (
            <Button
              variant="outline"
              onClick={() => setStep(step - 1)}
              className="px-6"
            >
              <ChevronLeft className="w-4 h-4 mr-1" />
              Back
            </Button>
          )}
          <Button
            onClick={handleNext}
            disabled={!canProceed()}
            className="flex-1 bg-gradient-to-r from-emerald-600 to-teal-600 hover:from-emerald-700 hover:to-teal-700"
          >
            {step === 4 ? 'Complete Setup' : 'Continue'}
            <ChevronRight className="w-4 h-4 ml-1" />
          </Button>
        </div>
      </div>
    </div>
  );
}